In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re

# Load dataset
data_path = "fra.txt"  # Replace with your file path
num_samples = 12000  # Number of sentence pairs to use

# Read file
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")

# Extract sentence pairs (English -> French)
eng_sentences, fr_sentences = [], []
for line in lines[:num_samples]:
    parts = line.split("\t")
    if len(parts) >= 2:
        eng_sentences.append(parts[0])
        fr_sentences.append("\t" + parts[1] + "\n")  # Add start (\t) and end (\n) tokens

# Function to clean text
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-ZÀ-ÿ?.!,¿' ]+", "", text)  # Remove non-alphabet characters
    return text

eng_sentences = [preprocess_text(s) for s in eng_sentences]
fr_sentences = [preprocess_text(s) for s in fr_sentences]

# Add start and end tokens to the French sentences
fr_sentences = ['\t' + sentence + '\n' for sentence in fr_sentences]



# Tokenization
def tokenize(sentences):
    tokenizer = Tokenizer(filters="")
    tokenizer.fit_on_texts(sentences)
    sequences = tokenizer.texts_to_sequences(sentences)
    return tokenizer, sequences

# Tokenize English and French sentences (including special tokens)
eng_tokenizer, eng_sequences = tokenize(eng_sentences)
fr_tokenizer, fr_sequences = tokenize(fr_sentences)

# Get vocabulary sizes
num_encoder_tokens = len(eng_tokenizer.word_index) + 1
num_decoder_tokens = len(fr_tokenizer.word_index) + 1

# Check if special tokens are added to the vocabulary
#print("Start token index:", fr_tokenizer.word_index['\t'])
#print("End token index:", fr_tokenizer.word_index['\n'])


# Get max sequence lengths
max_encoder_seq_length = max(len(seq) for seq in eng_sequences)
max_decoder_seq_length = max(len(seq) for seq in fr_sequences)

# Pad sequences (right padding)
encoder_input_data = pad_sequences(eng_sequences, maxlen=max_encoder_seq_length, padding="post")
decoder_input_data = pad_sequences(fr_sequences, maxlen=max_decoder_seq_length, padding="post")

# Create decoder target data (one-hot encoding)
decoder_target_data = np.zeros((num_samples, max_decoder_seq_length, num_decoder_tokens), dtype="float32")
for i, seq in enumerate(fr_sequences):
    for t, word_idx in enumerate(seq):
        if t > 0:  # Shift left by one step
            decoder_target_data[i, t - 1, word_idx] = 1.0
# LSTM model parameters
latent_dim = 256  # Latent dimensionality of the encoder/decoder LSTM

# Encoder
encoder_inputs = Input(shape=(max_encoder_seq_length,))
encoder_embedding = Embedding(num_encoder_tokens, latent_dim, mask_zero=True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(max_decoder_seq_length,))
decoder_embedding = Embedding(num_decoder_tokens, latent_dim, mask_zero=True)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define and compile the seq2seq model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])

# Train model
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=64, epochs=150, validation_split=0.2)

# Save model
model.save("eng_to_fr_large_model.h5")


# Inference Model - Encoder
encoder_model = Model(encoder_inputs, encoder_states)

# Inference Model - Decoder
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_embedding_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero=True)
decoder_embedding_inputs = decoder_embedding_layer(decoder_inputs)
decoder_outputs, state_h, state_c = decoder_lstm(decoder_embedding_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

# Reverse lookup dictionaries for decoding
reverse_eng_word_index = {v: k for k, v in eng_tokenizer.word_index.items()}
reverse_fr_word_index = {v: k for k, v in fr_tokenizer.word_index.items()}
fr_word_index = fr_tokenizer.word_index

# Function for sentence translation
def translate_sentence(sentence):
    sentence = preprocess_text(sentence)
    seq = eng_tokenizer.texts_to_sequences([sentence])
    padded_seq = pad_sequences(seq, maxlen=max_encoder_seq_length, padding="post")

    states_value = encoder_model.predict(padded_seq)

    # Initialize the target sequence with the start token
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = fr_word_index['\t']  # Start token

    translated_sentence = ""

    while True:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = reverse_fr_word_index.get(sampled_token_index, "")

        if sampled_word == "" or sampled_word == "\n" or len(translated_sentence.split()) > max_decoder_seq_length:
            break

        translated_sentence += " " + sampled_word
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]

    return translated_sentence.strip()

# Example translations
print(translate_sentence("hello"))  # Output: "bonjour"
print(translate_sentence("where is the bathroom?"))  # Output: "où sont les toilettes?"
print(translate_sentence("how are you?"))  # Output: "comment ça va?"


Epoch 1/150
150/150 ━━━━━━━━━━━━━━━━━━━━ 10s 52ms/step - accuracy: 0.0161 - loss: 5.7521 - val_accuracy: 0.0105 - val_loss: 5.0742
Epoch 2/150
150/150 ━━━━━━━━━━━━━━━━━━━━ 8s 40ms/step - accuracy: 0.0170 - loss: 4.5368 - val_accuracy: 0.0145 - val_loss: 4.9874
Epoch 3/150
150/150 ━━━━━━━━━━━━━━━━━━━━ 6s 41ms/step - accuracy: 0.0226 - loss: 4.4376 - val_accuracy: 0.0160 - val_loss: 4.9363
Epoch 4/150
150/150 ━━━━━━━━━━━━━━━━━━━━ 10s 40ms/step - accuracy: 0.0271 - loss: 4.3579 - val_accuracy: 0.0181 - val_loss: 4.8598
Epoch 5/150
150/150 ━━━━━━━━━━━━━━━━━━━━ 6s 41ms/step - accuracy: 0.0284 - loss: 4.3074 - val_accuracy: 0.0230 - val_loss: 4.7908
Epoch 6/150
150/150 ━━━━━━━━━━━━━━━━━━━━ 10s 41ms/step - accuracy: 0.0311 - loss: 4.2326 - val_accuracy: 0.0263 - val_loss: 4.7508
Epoch 7/150
150/150 ━━━━━━━━━━━━━━━━━━━━ 10s 40ms/step - accuracy: 0.0343 - loss: 4.1536 - val_accuracy: 0.0346 - val_loss: 4.6599
Epoch 8/150
150/150 ━━━━━━━━━━━━━━━━━━━━ 7s 44ms/step - accuracy: 0.0369 - loss: 4.092

In [ ]:
print(translate_sentence("i love you"))  # Output: ""

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
les !
 !
 !
 !
 !
 !
 !
 !
 !
 !
